In [3]:
import pickle
import pandas as pd
from ipywidgets import Label, Layout, FloatText, Checkbox, Button, Output, VBox, HBox, RadioButtons, interact_manual, BoundedFloatText, Dropdown, interactive_output


descrip_tyle = {'description_width': '140px'}
class_input = RadioButtons(options=['Bond', 'Credit'],
                           description='Treaty Class:',
                           style=descrip_tyle,
                           disabled=False)

commission_input = FloatText(value=0,
                             step=0.1,
                             description='Commission:',
                             style= descrip_tyle,
                             disabled=False)


prof_comm_input = FloatText(value=0,
                            step=0.1,
                            description='Profit Commission:',
                            style=descrip_tyle,
                            disabled=False)

broker_present_input = Checkbox(value=False,
                                description='Broker Present',
                                style=descrip_tyle,
                                disabled=False)

signed_input = BoundedFloatText(value=7.5,
                                min=0.01,
                                max=100,
                                step=0.01,
                                description='Signed %:',
                                style=descrip_tyle,
                                disabled=False)

terr_options = ['Africa',
                'Asia',
                'Emerging Europe',
                'Latin America',
                'Middle East & North Africa',
                'North America',
                'Pacific Region',
                'Western Europe']

terr_input = Dropdown(options=terr_options,
                      description='Territory:',
                      style=descrip_tyle,
                      disabled=False)


def out_prediction(treaty_class: str, signed: float, territory: str, commission: float = 0.0, prof_comm: float = 0.0, broker_present:bool = False):
    model = pickle.load(open('ml_model', 'rb'))
    df = pd.DataFrame(data=[[0] * len(model.feature_names_in_)],
                      columns=model.feature_names_in_)

    if treaty_class.startswith('C'):
        df.loc[0, 'IS_CRED'] = 1
    
    df.loc[0, 'Signed %'] = signed / 100

    df.loc[0, f'Terr_{territory}'] = 1
    
    df.loc[0, 'Commission'] = commission / 100
    
    df.loc[0, 'Prof Comm'] = prof_comm / 100

    if broker_present is True:
        df.loc[0, 'BROKER_PRESENT'] = 1

    print("{:.2%}".format(model.predict(df)[0]))

widget_out = interactive_output(
    out_prediction,
    {'treaty_class': class_input,
     'signed': signed_input,
     'territory': terr_input,
     'commission': commission_input,
     'prof_comm': prof_comm_input,
     'broker_present': broker_present_input}
)

items_layout = Layout( width='auto')     # override the default width of the button to 'auto' to let the button grow

box_layout = Layout(display='flex',
                    flex_flow='column',
                    align_items='stretch',
                    border='solid',
                    width='50%')

child_items = [class_input,
               commission_input,
               prof_comm_input,
               broker_present_input,
               signed_input,
               terr_input,
               widget_out]
box = VBox(children=child_items)
box